### ------------------------------------------------------------------------------------------------------------

In [ ]:
%pylab inline

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https: // github.com / impulsecorp / PickStocks.git
    !mv PickStocks / *.py.
    !mv PickStocks / data.
    !pip install -qq -r PickStocks / requirements.txt
    !pip install -qq scikit-optimize

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
from datablock import *
from backtesting import Backtest
seed

### ------------------------------------------------------------------------------------------------------------

In [ ]:
%%time

data_timeperiod = '60min'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
# data[0:20]
# print(get_data_features(data))
data.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_class = LogisticRegression

In [ ]:
system.train_set_end = 0.5
system.val_test_end = 0.7
system.max_tries = 50

clf_class = LogisticRegression
feature_name = featformat('move')

# True means it will trade only when abs(move) > threshold
# False means it will trade only when move > threshold
take_abs = True

In [ ]:
%%time

# search space boundaries
sp_min = 0.0
sp_max = 10.0
points = 50

bt = Backtest(data, MLSingleParamOverUnderStrategy, **btkw)
consts = dict(mode=['opt'], clf_class=[clf_class], period=[data_timeperiod],
              take_abs=[take_abs], feature_name=[feature_name], 
              direction=['above'])
optkw['method'] = 'skopt'
results = bt.optimize(**consts, **optkw,
                      threshold=np.linspace(sp_min, sp_max, points))
plot_optresult(get_optdata(results, consts), feature_name);

In [ ]:
# performance of best backtest
plot_result(bt, results)

In [ ]:
# stats for best backtest
results[0]

In [ ]:
gr = get_optdata(results, consts)
best_threshold = gr.index[np.argmax(gr.values)]
best_threshold, np.max(gr.values)

In [ ]:
%%time

# testing the optimized model on unseen data here
consts['mode'] = ['test']
stats = bt.run(**{x:consts[x][0] for x in consts.keys()},
               threshold=best_threshold)
display(stats)

In [ ]:
plot_result(bt, [stats])

### Same strategy but using an ensemble

In [ ]:
%%time

# search space boundaries
sp_min = 0.0
sp_max = 10.0
points = 50

MLEnsembleParamOverUnderStrategy.num_clfs = 100
MLEnsembleParamOverUnderStrategy.dropout = 0.05

bt = Backtest(data, MLEnsembleParamOverUnderStrategy, **btkw)
consts = dict(mode=['opt'], clf_class=[clf_class], period=[data_timeperiod],
              take_abs=[take_abs], feature_name=[feature_name], 
              direction=['above'])
optkw['method'] = 'skopt'
results = bt.optimize(**consts, **optkw,
                      threshold=np.linspace(sp_min, sp_max, points))
plot_optresult(get_optdata(results, consts), feature_name);

In [ ]:
# performance of best backtest
plot_result(bt, results)

In [ ]:
# stats for best backtest
results[0]

In [ ]:
gr = get_optdata(results, consts)
best_threshold = gr.index[np.argmax(gr.values)]
best_threshold, np.max(gr.values)

In [ ]:
%%time

# testing the optimized model on unseen data here
consts['mode'] = ['test']
stats = bt.run(**{x:consts[x][0] for x in consts.keys()},
               threshold=best_threshold)
display(stats)

In [ ]:
plot_result(bt, [stats])